<a href="https://colab.research.google.com/github/Strojove-uceni/23206-final-acoustic-emission-classification/blob/main/AE_classificatio_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# @title Imports and Helper Functions
%%capture
!pip install lightning
!pip3 install wandb

import torch
import torch.nn as nn
import torch.nn.functional as F
import pytorch_lightning as pl
import numpy as np
import pickle
import matplotlib.pyplot as plt
import pandas as pd
import wandb
import requests


from torch.utils.data import DataLoader, TensorDataset, random_split
from torch.utils.data import Dataset, DataLoader
from lightning.pytorch.loggers import WandbLogger
from sklearn.model_selection import train_test_split
from torchmetrics.classification import Accuracy, MulticlassConfusionMatrix
from pytorch_lightning.utilities.model_summary import ModelSummary
from torch import argmax
from sklearn.preprocessing import StandardScaler



In [2]:
from google.colab import drive
drive.mount('/content/drive')

!git clone https://github.com/Strojove-uceni/23206-final-acoustic-emission-classification.git


Mounted at /content/drive
Cloning into '23206-final-acoustic-emission-classification'...
remote: Enumerating objects: 19, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 19 (delta 4), reused 4 (delta 0), pack-reused 0
Receiving objects: 100% (19/19), 5.29 KiB | 5.29 MiB/s, done.
Resolving deltas: 100% (4/4), done.


In [5]:
path = '/content/23206-final-acoustic-emission-classification/Test_dataset_seed=43.pkl'
with open(path, 'rb') as pickle_file:
            loaded_data = pickle.load(pickle_file)
X_test = loaded_data['X_test']
X_test = np.moveaxis(X_test,1,2)
y_test = loaded_data['y_test']

In [6]:
X_test.shape

(1250, 4, 6000)